In [1]:
import pandas as pd

In [6]:
week1 = pd.read_csv('data/Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('data/Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('data/Restaurant - Customers.csv')
foods = pd.read_csv('data/Restaurant - Foods.csv')

# pd.concat()

In [9]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [10]:
week2.head(3)

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10


In [8]:
pd.concat([week1, week2], ignore_index=True)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


In [14]:
# not recomend way
week1.append(week2, ignore_index=True)

/var/folders/6t/jd7dl5tj4l52srlvl2vv4grh0000gn/T/ipykernel_48586/1486794766.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  week1.append(week2, ignore_index=True)


,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


# pd.concat() part2

In [18]:
# to keep the original index and use multi-index to distinguish
sales = pd.concat([week1, week2], keys=['week1', 'week2'])
sales.head(3)

Customer ID  Food ID
week1 0          537        9
      1           97        4
      2          658        1

In [19]:
sales.loc[('week1', 1)]

Customer ID    97
Food ID         4
Name: (week1, 1), dtype: int64

In [20]:
sales.loc[('week1',)]

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,413,9
246,926,6
247,134,3
248,396,6


# inner join, part 1

In [24]:
pd.merge(week1, week2, on='Customer ID', suffixes=(' - week1', ' - week2'))

,Customer ID,Food ID - week1,Food ID - week2
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9
...,...,...,...
57,945,5,4
58,343,3,5
59,343,3,2
60,343,3,7


# inner join - multiple column

In [27]:
temp = pd.merge(week1, week2, on='Customer ID')
mask = temp['Food ID_x'] == temp['Food ID_y']
temp[mask]

,Customer ID,Food ID_x,Food ID_y
16,304,3,3
24,540,3,3
26,937,10,10
28,233,3,3
29,21,4,4
30,21,4,4
37,922,1,1
54,578,5,5
55,578,5,5


In [28]:
pd.merge(week1, week2, on=['Customer ID', 'Food ID'])

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,21,4
6,922,1
7,578,5
8,578,5


# outer join

In [29]:
pd.merge(week1, week2, on='Customer ID', how='outer')

,Customer ID,Food ID_x,Food ID_y
0,537,9.0,5.0
1,97,4.0,NaN
2,658,1.0,NaN
3,202,2.0,NaN
4,155,9.0,3.0
...,...,...,...
449,855,NaN,4.0
450,559,NaN,10.0
451,276,NaN,4.0
452,556,NaN,10.0


In [31]:
merged = pd.merge(week1, week2, on='Customer ID', how='outer', indicator=True)
merged.head()

,Customer ID,Food ID_x,Food ID_y,_merge
0,537,9.0,5.0,both
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
4,155,9.0,3.0,both


In [32]:
merged['_merge'].value_counts()

right_only    197
left_only     195
both           62
Name: _merge, dtype: int64

In [36]:
mask = merged['_merge'] != 'both'
mask = merged['_merge'].isin(['left_only', 'right_only'])
merged[mask]

,Customer ID,Food ID_x,Food ID_y,_merge
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
6,213,8.0,NaN,left_only
7,600,1.0,NaN,left_only
...,...,...,...,...
449,855,NaN,4.0,right_only
450,559,NaN,10.0,right_only
451,276,NaN,4.0,right_only
452,556,NaN,10.0,right_only


# left join (similar to vlookup)

> use left as base and pull additional data from right if match

In [37]:
pd.merge(week1, week2, how='left', on='Customer ID')

,Customer ID,Food ID_x,Food ID_y
0,537,9,5.0
1,97,4,NaN
2,658,1,NaN
3,202,2,NaN
4,155,9,3.0
...,...,...,...
252,413,9,NaN
253,926,6,NaN
254,134,3,NaN
255,396,6,NaN


In [39]:
week1.head(2)

,Customer ID,Food ID
0,537,9
1,97,4


In [40]:
foods.head(3)

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99


In [38]:
pd.merge(week1, foods, how='left', on='Food ID')

,Customer ID,Food ID,Food Item,Price
0,537,9,Donut,0.99
1,97,4,Quesadilla,4.25
2,658,1,Sushi,3.99
3,202,2,Burrito,9.99
4,155,9,Donut,0.99
...,...,...,...,...
245,413,9,Donut,0.99
246,926,6,Pasta,13.99
247,134,3,Taco,2.99
248,396,6,Pasta,13.99


In [41]:
pd.merge(week1, foods, how='left', on='Food ID', sort=True)

,Customer ID,Food ID,Food Item,Price
0,658,1,Sushi,3.99
1,600,1,Sushi,3.99
2,155,1,Sushi,3.99
3,341,1,Sushi,3.99
4,20,1,Sushi,3.99
...,...,...,...,...
245,809,10,Drink,1.75
246,584,10,Drink,1.75
247,274,10,Drink,1.75
248,151,10,Drink,1.75


# left_on and right_on parameter

In [42]:
week2.head(3)

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10


In [43]:
customers.head(3)

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive


In [44]:
pd.merge(week2, customers, how='left', left_on='Customer ID', right_on='ID')

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,688,10,688,Carl,Williamson,Male,Thoughtmix,Graphic Designer
1,813,7,813,Johnny,Walker,Male,Kayveo,Developer II
2,495,10,495,Deborah,Little,Female,Babbleblab,VP Accounting
3,189,5,189,Roger,Gordon,Male,Skilith,Operator
4,267,3,267,Matthew,Wood,Male,Agimba,Product Engineer
...,...,...,...,...,...,...,...,...
245,783,10,783,Phyllis,Meyer,Female,Voolia,Information Systems Manager
246,556,10,556,Samuel,Bailey,Male,Oyoloo,Nurse
247,547,9,547,Tina,Watkins,Female,Thoughtstorm,Accountant II
248,252,9,252,Douglas,Powell,Male,Jetwire,Geologist IV


In [49]:
pd.merge(week2, customers, how='left', left_on='Customer ID', right_on='ID', sort=True).drop(['ID'], axis=1)

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,8,6,Frances,Adams,Female,Dabshots,Developer III
1,13,2,Ruth,Alvarez,Female,Twitterlist,Mechanical Systems Engineer
2,21,4,Albert,Burns,Male,Rhynoodle,Junior Executive
3,24,8,Donna,Thomas,Female,Jaxbean,Chief Design Engineer
4,27,4,Jessica,Bennett,Female,Twitternation,Account Executive
...,...,...,...,...,...,...,...
245,968,4,Teresa,Reynolds,Female,Flashdog,Budget/Accounting Analyst IV
246,969,3,Marie,Ortiz,Female,Meejo,Research Associate
247,977,7,Cynthia,Dixon,Female,Skalith,Automation Specialist II
248,994,2,Clarence,Morgan,Male,Edgewire,Geologist II


# merge by index

In [50]:
week1 = pd.read_csv('data/Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('data/Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('data/Restaurant - Customers.csv', index_col='ID')
foods = pd.read_csv('data/Restaurant - Foods.csv', index_col='Food ID')

In [51]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [52]:
customers.head(2)

,First Name,Last Name,Gender,Company,Occupation
ID,,,,,
1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer


In [53]:
foods.head(2)

,Food Item,Price
Food ID,,
1,Sushi,3.99
2,Burrito,9.99


In [58]:
# no additional "ID" column
sales = week1.merge(customers, how='left', left_on='Customer ID', right_index=True, sort=True)
sales

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
96,3,2,Roger,Black,Male,Tagfeed,Account Executive
143,10,2,Steven,Ryan,Male,Twinder,Community Outreach Specialist
31,20,1,Lisa,Rice,Female,Oloo,Programmer IV
101,21,4,Albert,Burns,Male,Rhynoodle,Junior Executive
212,21,4,Albert,Burns,Male,Rhynoodle,Junior Executive
...,...,...,...,...,...,...,...
11,966,5,Robert,Ford,Male,Jabbertype,Account Representative IV
175,968,1,Teresa,Reynolds,Female,Flashdog,Budget/Accounting Analyst IV
209,985,5,Julia,Ortiz,Female,Kwideo,Structural Analysis Engineer
194,991,2,Melissa,Wells,Female,Lazzy,Senior Sales Associate


In [60]:
sales.merge(foods, how='left', left_on='Food ID', right_index=True)

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation,Food Item,Price
96,3,2,Roger,Black,Male,Tagfeed,Account Executive,Burrito,9.99
143,10,2,Steven,Ryan,Male,Twinder,Community Outreach Specialist,Burrito,9.99
31,20,1,Lisa,Rice,Female,Oloo,Programmer IV,Sushi,3.99
101,21,4,Albert,Burns,Male,Rhynoodle,Junior Executive,Quesadilla,4.25
212,21,4,Albert,Burns,Male,Rhynoodle,Junior Executive,Quesadilla,4.25
...,...,...,...,...,...,...,...,...,...
11,966,5,Robert,Ford,Male,Jabbertype,Account Representative IV,Pizza,2.49
175,968,1,Teresa,Reynolds,Female,Flashdog,Budget/Accounting Analyst IV,Sushi,3.99
209,985,5,Julia,Ortiz,Female,Kwideo,Structural Analysis Engineer,Pizza,2.49
194,991,2,Melissa,Wells,Female,Lazzy,Senior Sales Associate,Burrito,9.99


In [61]:
week1.merge(week2, how='left', left_index=True, right_index=True)

,Customer ID_x,Food ID_x,Customer ID_y,Food ID_y
0,537,9,688,10
1,97,4,813,7
2,658,1,495,10
3,202,2,189,5
4,155,9,267,3
...,...,...,...,...
245,413,9,783,10
246,926,6,556,10
247,134,3,547,9
248,396,6,252,9


# .join()

In [63]:
week1 = pd.read_csv('data/Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('data/Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('data/Restaurant - Customers.csv')
foods = pd.read_csv('data/Restaurant - Foods.csv')

In [65]:
satisfaction = pd.read_csv('data/Restaurant - Week 1 Satisfaction.csv')
satisfaction.head(3)

,Satisfaction Rating
0,2
1,7
2,3


In [66]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [67]:
week1.merge(satisfaction, how='left', left_index=True, right_index=True)

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10
...,...,...,...
245,413,9,1
246,926,6,2
247,134,3,8
248,396,6,10


In [68]:
pd.concat([week1, satisfaction],axis='columns')

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10
...,...,...,...
245,413,9,1
246,926,6,2
247,134,3,8
248,396,6,10


In [71]:
# if two DF has the same index, you can simply use .join() to append column
# however, 'merge' is more general purpose and versatile
week1.join(satisfaction)

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10
...,...,...,...
245,413,9,1
246,926,6,2
247,134,3,8
248,396,6,10
